- Linear Regression을 통한 시장가치 예측 모형 코드입니다.
- 데이터 부족으로 Validation Set은 따로 생성하지 않았습니다.
- 연봉이 0인 데이터(미책정 선수)는 삭제해주었습니다.
- 전체 포지션 데이터의 경우 포지션을 one-hot-encoding 해주었습니다.
- 모델의 성능 평가 지표는 MAE, MSE, RMSE, MAPE, R^2 score를 활용했습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 및 라이브러리 불러오기

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np

In [30]:
data_path = '/content/drive/MyDrive/Colab Notebooks/캡스톤_4조/dataset/'
data = pd.read_csv(data_path + 'epl_data.csv')

data = data[data['연봉(€)']!=0]

data

,Unnamed: 0,이름,연봉(€),시즌,포지션,클럽팀,전체 출장 시간,전체 골,전체 어시스트,클린시트,...,90분당 골,90분당 실점,카드/90분,공격수 순위,미드필더 순위,수비수 순위,득점 순위,나이,시장가치,승점
0,0,João Moutinho,5992270.0,2022,Midfielder,wolverhamptonwanderers,508.0,0.0,0.0,2.0,...,0.00,0.89,0.35,108.0,121.0,-1.0,9.0,36,2.0,27
1,1,Matheus Nunes,5093430.0,2022,Midfielder,wolverhamptonwanderers,507.0,0.0,1.0,2.0,...,0.00,1.07,0.18,101.0,49.0,-1.0,8.0,24,45.0,27
2,2,Nélson Semedo,4609439.0,2022,Defender,wolverhamptonwanderers,263.0,0.0,0.0,2.0,...,0.00,0.68,0.34,-1.0,-1.0,-1.0,4.0,29,18.0,27
3,3,Daniel Podence,3595362.0,2022,Forward,wolverhamptonwanderers,503.0,2.0,0.0,3.0,...,0.36,1.25,0.18,26.0,226.0,-1.0,1.0,27,20.0,27
4,4,Rúben Neves,2996135.0,2022,Midfielder,wolverhamptonwanderers,720.0,1.0,0.0,3.0,...,0.13,1.00,0.63,89.0,225.0,-1.0,2.0,25,40.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,3193,Gabriel Paulista,2996135.0,2014,Defender,arsenal,349.0,0.0,1.0,1.0,...,0.00,0.77,0.00,284.0,29.0,14.0,27.0,24,12.0,75
3194,3194,Kieran Gibbs,2756444.0,2014,Defender,arsenal,1658.0,0.0,3.0,7.0,...,0.00,1.09,0.16,336.0,73.0,56.0,26.0,25,13.0,75
3195,3195,Calum Chambers,2696522.0,2014,Defender,arsenal,1569.0,1.0,1.0,8.0,...,0.06,1.03,0.52,199.0,190.0,44.0,14.0,19,12.0,75
3196,3196,Francis Coquelin,1797681.0,2014,Midfielder,arsenal,1663.0,0.0,0.0,7.0,...,0.00,0.65,0.27,343.0,371.0,-1.0,21.0,23,3.0,75


연봉이 0인 데이터 104개가 삭제

In [31]:
# 결측치 확인 및 대체
data[data['시장가치'].isnull()]
data = data.fillna(2)

In [32]:
data[data['시장가치'].isnull()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3095 entries, 0 to 3197
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3095 non-null   int64  
 1   이름          3095 non-null   object 
 2   연봉(€)       3095 non-null   float64
 3   시즌          3095 non-null   int64  
 4   포지션         3095 non-null   object 
 5   클럽팀         3095 non-null   object 
 6   전체 출장 시간    3095 non-null   float64
 7   전체 골        3095 non-null   float64
 8   전체 어시스트     3095 non-null   float64
 9   클린시트        3095 non-null   float64
 10  실점          3095 non-null   float64
 11  경고          3095 non-null   float64
 12  퇴장          3095 non-null   float64
 13  90분당 골 관여   3095 non-null   float64
 14  90분당 어시스트   3095 non-null   float64
 15  90분당 골      3095 non-null   float64
 16  90분당 실점     3095 non-null   float64
 17  카드/90분      3095 non-null   float64
 18  공격수 순위      3095 non-null   float64
 19  미드필더 순위     3095 non-null  

In [33]:
# 데이터 불러오는 함수
def load_data(Position):
  data_path = '/content/drive/MyDrive/Colab Notebooks/캡스톤_4조/dataset/'
  data = pd.read_csv(data_path + 'epl_{}_data.csv'.format(Position))

  del data['Unnamed: 0']
  del data['이름'], data['클럽팀']
  data = data[data['연봉(€)']!=0]

  return data

In [34]:
# 포지션 one-hot-encoding
def change(x):
    cat = ''
    if x == 'Goalkeeper': cat = 0
    elif x == 'Defender': cat = 1
    elif x == 'Midfielder': cat = 2
    elif x == 'Forward': cat = 3
    return cat

data['포지션'] = data.포지션.apply(lambda x: change(x))

del data['Unnamed: 0']
del data['이름'], data['클럽팀']

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3095 entries, 0 to 3197
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연봉(€)      3095 non-null   float64
 1   시즌         3095 non-null   int64  
 2   포지션        3095 non-null   int64  
 3   전체 출장 시간   3095 non-null   float64
 4   전체 골       3095 non-null   float64
 5   전체 어시스트    3095 non-null   float64
 6   클린시트       3095 non-null   float64
 7   실점         3095 non-null   float64
 8   경고         3095 non-null   float64
 9   퇴장         3095 non-null   float64
 10  90분당 골 관여  3095 non-null   float64
 11  90분당 어시스트  3095 non-null   float64
 12  90분당 골     3095 non-null   float64
 13  90분당 실점    3095 non-null   float64
 14  카드/90분     3095 non-null   float64
 15  공격수 순위     3095 non-null   float64
 16  미드필더 순위    3095 non-null   float64
 17  수비수 순위     3095 non-null   float64
 18  득점 순위      3095 non-null   float64
 19  나이         3095 non-null   int64  
 20  시장가치    

In [35]:
GK = load_data('GK')
DF = load_data('DF')
MF = load_data('MF')
FW = load_data('FW')

GK # check

,연봉(€),시즌,포지션,전체 출장 시간,전체 골,전체 어시스트,클린시트,실점,경고,퇴장,...,90분당 골,90분당 실점,카드/90분,공격수 순위,미드필더 순위,수비수 순위,득점 순위,나이,시장가치,승점
0,5992270.0,2022,Goalkeeper,720.0,0.0,0.0,2.0,10.0,0.0,0.0,...,0.0,1.25,0.00,216.0,198.0,45.0,13.0,36,7.0,71
2,5992270.0,2022,Goalkeeper,720.0,0.0,0.0,1.0,21.0,1.0,0.0,...,0.0,2.63,0.13,235.0,236.0,95.0,24.0,25,22.0,27
3,3595362.0,2022,Goalkeeper,720.0,0.0,0.0,3.0,7.0,0.0,0.0,...,0.0,0.88,0.00,219.0,227.0,13.0,8.0,30,18.0,73
4,22471014.0,2022,Goalkeeper,630.0,0.0,0.0,2.0,14.0,0.0,0.0,...,0.0,2.00,0.00,140.0,131.0,86.0,17.0,32,15.0,67
5,5992270.0,2022,Goalkeeper,720.0,0.0,0.0,4.0,8.0,0.0,0.0,...,0.0,1.00,0.00,141.0,110.0,23.0,14.0,29,45.0,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,5992270.0,2014,Goalkeeper,2814.0,0.0,0.0,13.0,28.0,1.0,0.0,...,0.0,0.90,0.03,389.0,393.0,24.0,14.0,22,25.0,87
243,898841.0,2014,Goalkeeper,3420.0,0.0,0.0,10.0,51.0,1.0,0.0,...,0.0,1.34,0.03,380.0,385.0,123.0,16.0,28,1.2,33
244,3115981.0,2014,Goalkeeper,270.0,0.0,0.0,1.0,7.0,1.0,0.0,...,0.0,2.33,0.33,-1.0,-1.0,-1.0,15.0,38,1.0,38
245,1797681.0,2014,Goalkeeper,3060.0,0.0,0.0,9.0,44.0,2.0,0.0,...,0.0,1.29,0.06,359.0,366.0,110.0,11.0,30,5.0,38


# Data split

In [36]:
# 데이터 분할 함수
def data_split(df):
  train_df = df[df['시즌']!=2022]
  test_df = df[df['시즌']==2022]

  train_X, train_y = train_df.drop('시장가치', axis=1), train_df['시장가치']
  test_X, test_y = test_df.drop('시장가치', axis=1), test_df['시장가치']

  return train_X, train_y, test_X, test_y

In [37]:
train_X, train_y, test_X, test_y = data_split(data)

# Linear Regression

In [38]:
# Pipeline
pipeline = Pipeline([
    ("MinMax Scaling", MinMaxScaler()),
    ("LinearRegression", LinearRegression())
])

In [39]:
pipeline.fit(train_X, train_y)

Pipeline(steps=[('MinMax Scaling', MinMaxScaler()),
                ('LinearRegression', LinearRegression())])

In [40]:
pred_y = pipeline.predict(test_X)

In [ ]:
# def MAPE(y_test, y_pred):
# 	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

# MAE = mean_absolute_error(test_y, pred_y)
# MSE = mean_squared_error(test_y, pred_y)
# RMSE = np.sqrt(MSE)
# MAPE = MAPE(test_y, pred_y)

# print('MAE: ', MAE)
# print('MSE: ', MSE)
# print('RMSE: ', RMSE)
# print('MAPE', MAPE)
# print('Score', pipeline.score(test_X, test_y))

MAE:  2984478.242171041
MSE:  21430645293076.54
RMSE:  4629324.496411603
MAPE 72.7780148641236
Score 0.10496661539406471


In [41]:
def MAPE(y_test, y_pred):
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

MAE = mean_absolute_error(test_y, pred_y)
MSE = mean_squared_error(test_y, pred_y)
RMSE = np.sqrt(MSE)
MAPE = MAPE(test_y, pred_y)

print('MAE: ', MAE)
print('MSE: ', MSE)
print('RMSE: ', RMSE)
print('MAPE', MAPE)
print('Score', pipeline.score(test_X, test_y))

MAE:  11.52923103643259
MSE:  259.1251073199374
RMSE:  16.097363365468812
MAPE 62.621236130763435
Score 0.3661214539455586
